In [7]:
#from joblib import Parallel, delayed
#import joblib
#import pandas as pd
#import numpy as np

In [8]:
#RF_tuned_from_joblib = joblib.load('RF_clf_tuned.pkl')

In [26]:
def classification_text(way):
    #importing libraries
    
    from joblib import Parallel, delayed
    import joblib
    
    import pandas as pd
    import numpy as np
    
    import string
    import re
    
    from pymorphy2 import MorphAnalyzer
    
    import nltk
    nltk.download("stopwords")
    from nltk.corpus import stopwords
    from string import punctuation
    russian_stopwords = stopwords.words("russian")
    nltk.download('punkt')
    
    from tqdm import tqdm
    
    from nltk import word_tokenize
    
    #ML_model
    
    RF_tuned_from_joblib = joblib.load('RF_clf_tuned.pkl')
    
    
    # text preprocessing functiona
    
    def remove_punctuation(text):
        return "".join([ch if ch not in punctuation_new else ' ' for ch in text])
    def remove_numbers(text):
        return ''.join([i if not i.isdigit() else ' ' for i in text])
    def remove_multiple_spaces(text):
        return re.sub(r'\s+', ' ', text, flags=re.I)
    
    #New punctuations list for exclusion
    punctuation_new=re.sub("_","",punctuation)
    
    #Русские стоп-слова, которые надо будет удалить.
    f='которых которые твой которой которого сих ком свой твоя этими слишком нами всему будь саму чаще ваше сами наш затем еще самих наши ту каждое мочь весь этим наша своих оба который зато те этих вся ваш такая теми ею которая нередко каждая также чему собой самими нем вами ими откуда такие тому та очень сама нему алло оно этому кому тобой таки твоё каждые твои мой нею самим ваши ваша кем мои однако сразу свое ними всё неё тех хотя всем тобою тебе одной другие эта о само эта буду самой моё своей такое всею будут своего кого свои мог нам особенно её самому наше кроме вообще вон мною никто это'
    f=f.split()
    russian_stopwords = stopwords.words("russian")
    russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
    russian_stopwords.extend(f)###
    russian_stopwords.extend('весь')###
    russian_stopwords.extend(['звоню','звонить','звонила','говорю', 'банк','говорить', 'сбер', 
                         'сбербанк'])####
    
    # ф-я удалябщая стоп-слова
    def remove_stop_words(text):
        tokens = word_tokenize(text) 
        tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
        return " ".join(tokens)
    
    #ф-я лемматизации
    pymorphy = MorphAnalyzer()
    def lemmatize_with_pymorphy(tokens):
        lemms = [pymorphy.parse(token)[0].normal_form for token in tokens]
        return lemms
    
  
    #data manipulations
    
    data=pd.read_csv(way)
    preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
    data['preprocessed'] = list(map(preproccessing, data['text_employer']))
    
    #cresting text vector of X's
    sw_texts_list = []
    for text in tqdm(data['preprocessed']):
        tokens = word_tokenize(text)
        tokens = lemmatize_with_pymorphy(tokens)
        tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
        text = " ".join(tokens)
        sw_texts_list.append(text)
    data['text_lemma_with_underscore'] = sw_texts_list
    
    
    data_final=data.copy()
    data_final['predictions_labels']=RF_tuned_from_joblib.predict(data['text_lemma_with_underscore'])
    
    # Create new column using dictionary
    new_data = { 'Бизнес-карта':1,'Зарплатные проекты':2,'Открытие банковского счета':3,'Эквайринг':4}
    data_final['код']=data_final['predictions_labels'].map(new_data)
    data_final_answer=data_final[['ucid','text_employer','код']]
    data_final_answer.to_excel("function_classification_results.xlsx", sheet_name='Results') 
    
    return data_final_answer

In [27]:
way_1='/Users/vladimirant/Downloads/topic_modeling_task_sample_trainPart.csv'

In [28]:
classification_text(way_1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladimirant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladimirant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 42765/42765 [30:58<00:00, 23.01it/s]  


,ucid,text_employer,код
0,"5,01000641613474E+017",добрый_день. это клиентский_менеджер виктория ...,1
1,"5,01001121614156E+017",добрый_день. меня зовут клиентский_менеджер_сб...,2
2,"5,01001811614178E+017",добрый_день. меня зовут сбербанк блок сбер биз...,1
3,"5,01002531614256E+017",еще раз добрый_день клиентский_менеджер удобно...,1
4,"5,01005041613457E+017",добрый_день. это сбербанк блок сбер бизнес кли...,2
...,...,...,...
42760,"5,01034111613466E+017",добрый_день. это ваш персональный_менеджер_сбе...,2
42761,"5,01034111613466E+017",здравствуйте. как я могу к вам обращаться. на ...,2
42762,"5,01035251614262E+017",добрый_день. персональный_менеджер_сбербанк ва...,3
42763,"5,01035571614254E+017",добрый_день. вас персональный_менеджер сбер би...,1
